In [5]:
%config IPCompleter.use_jedi = False
%pdb off
%load_ext autoreload
%autoreload 3

# required to enable non-blocking interaction:
%gui qt5

import sys
from copy import deepcopy
from typing import Dict, List, Tuple, Optional, Callable, Union, Any
from typing_extensions import TypeAlias
from nptyping import NDArray
import neuropy.utils.type_aliases as types
from collections import defaultdict

import numpy as np
import pandas as pd

# Set up numpy print options to only wrap at window width:
np.set_printoptions(edgeitems=30, linewidth=100000, formatter=dict(float=lambda x: "%g" % x)) # this somehow fucked up and caused rounding of my values...


# Plotting
# import pylustrator # customization of figures
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
_bak_rcParams = mpl.rcParams.copy()

matplotlib.use('Qt5Agg')
# %matplotlib inline
# %matplotlib widget
# %matplotlib auto

# Switch to the desired interactivity mode
plt.interactive(True)
from mpl_multitab import MplMultiTab, MplMultiTab2D ## Main imports
from neuropy.utils.matplotlib_helpers import TabbedMatplotlibFigures
from pyphoplacecellanalysis.Pho2D.matplotlib.CustomMatplotlibTabbedWidget import CustomMatplotlibTabbedWidget

from pyphocorehelpers.indexing_helpers import get_bin_centers


Automatic pdb calling has been turned OFF
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import numpy as np

def generate_random_numpy_arrays(n: int, shape: tuple, dtype: type = float, seed: int = None):
    """
    Generates a list of `n` numpy arrays filled with random numbers.

    Args:
        n (int): Number of arrays to generate.
        shape (tuple): Shape of each array.
        dtype (type): Data type of the arrays. Default is float.
        seed (int): Optional random seed for reproducibility.

    Returns:
        list[np.ndarray]: List of numpy arrays filled with random numbers.
    
    Usage:
        arrays = generate_random_numpy_arrays(5, (2, 3), float, seed=42)
    """
    if seed is not None:
        np.random.seed(seed)
    return [np.random.rand(*shape).astype(dtype) for _ in range(n)]


def _perform_plot(x_data, y_data, extant_fig=None, extant_axes=None, **kwargs):
    if extant_fig is not None:
        fig = extant_fig # use the existing passed figure
        # fig.set_size_inches([23, 9.7])
    else:
        if extant_axes is None:
            fig = plt.figure(figsize=(23, 9.7))
        else:
            fig = extant_axes.get_figure()
            

    if extant_axes is not None:
        # user-supplied extant axes [axs0, axs1]
        assert isinstance(extant_axes, dict), f"extant_axes should be a dict but is instead of type: {type(extant_axes)}\n\textant_axes: {extant_axes}"
        assert len(extant_axes) == 1
        ax_scatter = extant_axes["ax_scatter"]
    else:
        # Need axes:
        # Layout Subplots in Figure:
        extant_axes = fig.subplot_mosaic(
            [   
                ["ax_scatter"],
            ],
        )
        ax_scatter = extant_axes["ax_scatter"]
        # gs = fig.add_gridspec(1, 8)
        # gs.update(wspace=0, hspace=0.05) # set the spacing between axes. # `wspace=0`` is responsible for sticking the pf and the activity axes together with no spacing
        # ax_activity_v_time = fig.add_subplot(gs[0, :-1]) # all except the last element are the trajectory over time
        # ax_pf_tuning_curve = fig.add_subplot(gs[0, -1], sharey=ax_activity_v_time) # The last element is the tuning curve
        # ax_pf_tuning_curve.set_title('Normalized Placefield', fontsize='14')
        ax_scatter.set_xticklabels([])
        ax_scatter.set_yticklabels([])
    # end else
    
    ## BEGIN FUNCTION BODY
    ax_scatter.plot(x_data, y_data)
    
    return fig, extant_axes



In [7]:


# Example usage
num_plots: int = 7
arrays = generate_random_numpy_arrays(num_plots, (25, ), float, seed=42)
plot_data_dict = {f"arr[{i}]":arr for i, arr in enumerate(arrays)}
plot_data_dict
## INPUTS: arrays, plot_data_dict

plot_subplot_mosaic_dict = {f"arr[{i}]":dict(sharex=True, sharey=True,) for i, arr in enumerate(arrays)}
ui, figures_dict, axs_dict = TabbedMatplotlibFigures.build_tabbed_multi_figure(plot_subplot_mosaic_dict)

for a_name, ax in axs_dict.items():
	plot_arr = plot_data_dict[a_name]
	fig = figures_dict[a_name]
	fig, extant_axes = _perform_plot(x_data=np.arange(len(plot_arr)), y_data=plot_arr, extant_fig=fig, extant_axes=ax)
ui.show()